In [9]:
from pyspark.sql import SparkSession
from operator import add

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.113:7077") \
        .appName("esraa_mohammed_a")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("INFO")


#A.1.1 :

In [10]:
elines = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.fi-en.en")
elines.count()


1924942

#A.1.2:

In [11]:
flines = spark_context.textFile("hdfs://192.168.2.113:9000/europarl/europarl-v7.fi-en.fi")
flines.count()

1924942

#A.1.3 :
total number of lines for both languages are : 1924942

#A.1.4: 3 partitions

In [12]:
print(elines.getNumPartitions())
print(flines.getNumPartitions())

3
3


#A.2.1  and #A.3.1

In [13]:
def preprocess(lines):
    lower = lines.map(lambda line : line.lower())\
                 .flatMap(lambda s: s.split(' '))\
                 .flatMap(lambda w: w.split('\n'))\
                 .map(lambda w: w.strip())\
                 .map(lambda w: (w,1))\
                 .reduceByKey(add)\
                 .takeOrdered(10, key=lambda x: -x[1])
    return lower


#A.2.2

In [14]:
print(preprocess(elines))

[('the', 3631865), ('of', 1724191), ('to', 1600135), ('and', 1339070), ('in', 1127220), ('that', 830098), ('a', 803908), ('is', 785410), ('for', 553730), ('we', 548087)]


In [15]:
print(preprocess(flines))

[('ja', 1249156), ('on', 1035956), ('että', 619655), ('euroopan', 257568), ('ei', 246268), ('myös', 178765), ('ovat', 161869), ('se', 152857), ('arvoisa', 149589), ('ole', 134745)]


#A.2.3

In [16]:
print(flines.count())
print(elines.count())

1924942
1924942


#A.3.2 :

Most of the frequently used words in english are articles, pronounce, propositions and verb to be. same in French language

In [ ]:
elines.map(lambda line: line.split(' ')).map(lambda w: len(w) < 6).take(5)

In [21]:
elines.take(5)

['Resumption of the session',
 'I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.',
 "Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful.",
 'You have requested a debate on this subject in the course of the next few days, during this part-session.',
 "In the meantime, I should like to observe a minute' s silence, as a number of Members have requested, on behalf of all the victims concerned, particularly those of the terrible storms, in the various countries of the European Union."]